<a href="https://colab.research.google.com/github/Tincho1902/PosgradoIA/blob/main/3b_Custom_embedding_con_Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto.

In [1]:
 ! pip install -q kaggle

In [2]:
from google.colab import files

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"martinfmartinez","key":"7b8eb549de5abdf931ebc9de96474b3a"}'}

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import platform
import multiprocessing
from gensim.models import Word2Vec

### Datos
Seleccionaremos reseñas o críticas de restaurantes como nuestro conjunto de datos.

In [11]:
dataset_name = 'joebeachcapital/restaurant-reviews'
zip_name = dataset_name.split('/')[-1]

!kaggle datasets download -d {dataset_name}
!unzip -q ./{zip_name}.zip

 82% 1.00M/1.23M [00:01<00:00, 1.04MB/s]
100% 1.23M/1.23M [00:01<00:00, 1.23MB/s]


In [12]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs

filename = 'Restaurant reviews.csv'
df_orig = pd.read_csv(filename, header=0)
df = df_orig['Review'].to_frame()  # Dejamos solamente los reviews

In [13]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 10000


In [14]:
# Filter rows where 'column_name' contains float entries
df = df[~df['Review'].apply(lambda x: isinstance(x, float))]

# Reset the index if needed
df.reset_index(drop=True, inplace=True)

In [15]:
df

,Review
0,"The ambience was good, food was quite good . h..."
1,Ambience is too good for a pleasant evening. S...
2,A must try.. great food great ambience. Thnx f...
3,Soumen das and Arun was a great guy. Only beca...
4,Food is good.we ordered Kodi drumsticks and ba...
...,...
9950,Madhumathi Mahajan Well to start with nice cou...
9951,This place has never disappointed us.. The foo...
9952,"Bad rating is mainly because of ""Chicken Bone ..."
9953,I personally love and prefer Chinese Food. Had...


### 1 - Preprocesamiento

In [16]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df.iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [17]:
# Demos un vistazo
sentence_tokens[:2]

[['the',
  'ambience',
  'was',
  'good',
  'food',
  'was',
  'quite',
  'good',
  'had',
  'saturday',
  'lunch',
  'which',
  'was',
  'cost',
  'effective',
  'good',
  'place',
  'for',
  'a',
  'sate',
  'brunch',
  'one',
  'can',
  'also',
  'chill',
  'with',
  'friends',
  'and',
  'or',
  'parents',
  'waiter',
  'soumen',
  'das',
  'was',
  'really',
  'courteous',
  'and',
  'helpful'],
 ['ambience',
  'is',
  'too',
  'good',
  'for',
  'a',
  'pleasant',
  'evening',
  'service',
  'is',
  'very',
  'prompt',
  'food',
  'is',
  'good',
  'over',
  'all',
  'a',
  'good',
  'experience',
  'soumen',
  'das',
  'kudos',
  'to',
  'the',
  'service']]

### 2 - Crear los vectores (word2vec)

In [18]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [19]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [20]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [21]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9955


In [22]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 4532


### 3 - Entrenar embeddings

In [23]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 3726400.0
Loss after epoch 1: 2816505.5
Loss after epoch 2: 2709011.5
Loss after epoch 3: 2594399.0
Loss after epoch 4: 2566188.0
Loss after epoch 5: 2534682.0
Loss after epoch 6: 2437466.0
Loss after epoch 7: 2412554.0
Loss after epoch 8: 2393078.0
Loss after epoch 9: 2374238.0
Loss after epoch 10: 2360998.0
Loss after epoch 11: 2345646.0
Loss after epoch 12: 2329310.0
Loss after epoch 13: 2417608.0
Loss after epoch 14: 2405736.0
Loss after epoch 15: 2388608.0
Loss after epoch 16: 2383148.0
Loss after epoch 17: 2367892.0
Loss after epoch 18: 2363280.0
Loss after epoch 19: 2357312.0


(7019706, 10036140)

### 4 - Ensayar

In [53]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["peaceful"], topn=10)

[('calm', 0.6981967091560364),
 ('cozy', 0.6787337064743042),
 ('funky', 0.6740900278091431),
 ('comfy', 0.6643236875534058),
 ('lite', 0.6601787805557251),
 ('cosy', 0.6571520566940308),
 ('lively', 0.6561968326568604),
 ('noisy', 0.6534141302108765),
 ('classy', 0.653216540813446),
 ('relaxed', 0.6503607034683228)]

In [54]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["peaceful"], topn=10)

[('mentioned', 0.02148730680346489),
 ('suey', 0.017179016023874283),
 ('gst', 0.0025229614693671465),
 ('clearly', -0.0008670181850902736),
 ('forgot', -0.001703247195109725),
 ('recipe', -0.005532057024538517),
 ('instructions', -0.013668016530573368),
 ('incorrect', -0.018014127388596535),
 ('carrot', -0.018223727121949196),
 ('via', -0.01888544112443924)]

In [29]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["food"], topn=10)

[('ambeince', 0.5751393437385559),
 ('cusine', 0.5480858683586121),
 ("food's", 0.5458176136016846),
 ('foods', 0.5444780588150024),
 ('timely', 0.5394572615623474),
 ('sevice', 0.5349171757698059),
 ('economic', 0.5331569910049438),
 ('👍👍', 0.5240759253501892),
 ('fare', 0.5222740769386292),
 ('lightning', 0.5211569666862488)]

In [48]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["chef"], topn=15)

[('pankaj', 0.5231897830963135),
 ('salim', 0.5226008296012878),
 ('subendu', 0.5213561654090881),
 ('vijay', 0.5184333920478821),
 ('manager', 0.5024229288101196),
 ('hema', 0.5023354887962341),
 ('aritra', 0.49766644835472107),
 ('shoutout', 0.49628639221191406),
 ('singh', 0.4932103455066681),
 ('ali', 0.4919898509979248),
 ('nitish', 0.4891344904899597),
 ('dharma', 0.4861096739768982),
 ('allauddin', 0.48603832721710205),
 ('himself', 0.48521414399147034),
 ('pradeep', 0.4818795323371887)]

El modelo relaciona la palabra 'CHEF' con los nombres de los encargados de los restaurant

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

In [43]:
# el método `get_vector` permite obtener los vectores:
vector_ambience = w2v_model.wv.get_vector("ambience")
print(vector_ambience)

[ 4.14231032e-01  1.25073224e-01 -1.26228109e-01 -1.52652666e-01
  2.82998942e-02 -3.52003872e-01  1.69325367e-01 -1.14752017e-01
  1.42177314e-01 -5.10612071e-01  4.62383419e-01  8.64016339e-02
 -6.05011545e-02  5.32153212e-02 -2.83586010e-02  8.82808864e-02
  1.95855677e-01  1.80774614e-01 -3.53026241e-01 -9.94915888e-02
 -7.85724968e-02  1.88728571e-01 -4.04096693e-02 -4.30307677e-03
  4.99009453e-02 -5.09971797e-01 -1.92587838e-01 -6.31135628e-02
 -4.18882489e-01 -4.10037398e-01  1.06200710e-01 -1.70317199e-02
 -1.81330532e-01 -1.04347572e-01  2.06521839e-01  1.33665055e-01
 -1.08799882e-01  3.69970798e-02 -2.83134729e-01  1.10480696e-01
  2.22274717e-02  7.69096836e-02 -1.13155954e-01 -1.23962618e-01
  2.51072764e-01 -4.14035954e-02  4.04011816e-01 -5.92946820e-03
 -6.23182058e-02 -1.27890751e-01  1.71601012e-01 -1.34159163e-01
 -9.40172598e-02  2.72445291e-01  8.00573677e-02  1.78843006e-01
  1.41722113e-01 -5.23242727e-02  2.30035409e-01  2.21060395e-01
  1.05034083e-01 -4.23707

In [44]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_ambience)

[('ambience', 0.9999999403953552),
 ('ambeince', 0.6245776414871216),
 ('lightning', 0.6185626983642578),
 ('ambiance', 0.5755875110626221),
 ('sevice', 0.5739502310752869),
 ('economic', 0.5628871321678162),
 ('funky', 0.5548626780509949),
 ('djs', 0.5432112812995911),
 ('dimly', 0.5395596623420715),
 ('setting', 0.5307633876800537)]

El modelo identifica una fuerte relación entre las palabras 'ambience', 'ambeince' y 'ambiance', incluso a pesar de que estas palabras están mal escritas. Estas tres palabras son consideradas las más relevantes en relación con el término 'ambience'.

In [46]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('loves', 0.5349171161651611),
 ('rocks', 0.45880383253097534),
 ('addicted', 0.4444737732410431),
 ('prefer', 0.4437088072299957),
 ('hello', 0.4374333918094635),
 ('💕', 0.43464502692222595),
 ('😍😍', 0.4299427568912506),
 ('bet', 0.42912647128105164),
 ('loved', 0.427810400724411),
 ('herb', 0.42758408188819885)]

Este caso se destaca como el más representativo, ya que la mayoría de las palabras relacionadas con "LOVE" expresan un sentimiento de agrado o gusto, lo que subraya la fuerte asociación de la palabra "LOVE" con emociones positivas.

### 5 - Visualizar agrupación de vectores

In [36]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [37]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [38]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [39]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Obtener conclusiones.

> El modelo tiene la capacidad de representar cada palabra presente en el corpus en un espacio donde palabras con significados similares se agrupan cerca unas de otras, mientras que palabras con significados opuestos tienden a ubicarse en regiones opuestas del espacio. Es importante señalar que todas estas relaciones se derivan de las opiniones expresadas por las personas que han escrito sobre el restaurante que visitaron. Sin embargo, es esencial recordar que estas relaciones son específicas al contexto de las críticas y pueden no mantenerse en otros contextos, donde las palabras pueden tener interpretaciones diferentes.

